# Building a Recurrent Neural Network

## Sentiment Analysis
In this project, we will build a Long Short-term Memory (LSTM) neural network to solve a binary sentiment analysis problem.

For this, we'll use the “IMDB Movie Review Dataset" available on Keras. It includes 50000 highly polarized movie reviews categorized as positive or negative.

## Importing the required libraries
We'll start with importing required libraries.

📌 Use the keyword "import".

In [3]:
!pip install tensorflow


In [4]:
# Import TensorFlow
import tensorflow as tf

# Import NumPy and Matplotlib
import numpy as np
import matplotlib.pyplot as plt

## Dataset
Let's download the IMDB dataset which is included in Keras, and assign it to the corresponding variables *X_train*, *y_train*, *X_test*, and *y_test*. We want to include the most frequently used 10000 words, so we specify 10000 for the num_words parameter.

📌 Use the datasets.imdb.load_data() function of the Keras.

In [5]:
# Download the IMDB dataset included in Keras
# Set the parameter num_words to 10000
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=10000)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Bu veri setini TensorFlow'dan indirirken ayarlayabileceğimiz başka bir parametre daha var; num_words. Bu parametre için 'n' sayısını belirtirsek, yalnızca en sık kullanılan 'n' kelime indirilecek, aksi takdirde tüm kelimeler tutulacaktır. En sık kullanılan 10000 kelimeyi dahil etmek istiyoruz. Elbette, daha yüksek bir sayı seçebiliriz ancak bunu yapmak çok fazla bellek gerektirecektir.

Before we move on, we can print a single sample to see what the data looks like.

📌 Use the print() function for this.

In [ ]:
# Print a sample
print(X_train[0])

Then, we print the the number of samples in the X_train and X_test datasets to see how the dataset is distributed.

📌 Use f-strings for this.

 Her listenin farklı bir film eleştirisine karşılık geldiği bir liste dizisine sahip olduğumuzu görebiliriz. Bu listelerin içinde çok sayıda sayı görüyoruz. Gerçek film incelemeleri kelimelerden oluşuyordu, ancak dize verilerinde matematiksel işlemler yapamayacağımız için bu kelimelerin sayılara dönüştürülmesi gerekiyordu. Neyse ki bu bizim için zaten yapılmıştı; her kelime veri kümesindeki belirli bir sayıya eşlendi. Örneğin "iyi" kelimesi 49 sayısına, "kötü" kelimesi 75 sayısına eşlendi. Bu varsayılan olarak yapılmamış olsaydı, kelimelere sayı atadığımız bir betik kolayca oluşturabilirdik.

In [7]:
# Print the number of samples
print(f"X_train: {len(X_train)}")
print(f"X_train: {len(X_test)}")

X_train: 25000
X_train: 25000


# Preprocessing
### Concatenate

To split the dataset with 80-10-10 ratio, we'll first concatenate train and test datasets to create one big dataset.

📌 Use contenate() function of the NumPy library for this.

In [9]:
# Concatenate X_train and X_test and assing it to a variable X
X = np.concatenate((X_train, X_test), axis=0)

# Concatenate y_train and y_test and assing it to a variable y
y = np.concatenate((y_train, y_test),axis=0)

###Padding

Since all reviews are at different lengths, we'll use padding to make all of them same length.

📌 Use preprocessing.sequence.pad_sequences() function for this.

In [10]:
# Pad all reviews in the X dataset to the length maxlen=1024
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=1024)

Bu veri kümelerini bölmeden önce, bazı ön işleme işlemleri yapabiliriz, böylece daha sonra her veri kümesi için bunu üç kez yapmamıza gerek kalmaz. Şimdi yapacağımız şeye 'dolgu' denir. Film incelemeleri farklı uzunluklarda olduğundan, tıpkı CNN'lerde yaptığımız gibi incelemelerin başına birkaç sıfır ekleyebiliriz, böylece tüm örnekler aynı uzunlukta olur. Buradaki max_len parametresi tüm incelemeleri 1024 kelimeye yeniden boyutlandıracaktır. Bu uzunluğun altındaki tüm örnekler ekstra sıfırlar alacak ve bu uzunluğun üzerindeki tüm incelemeler kırpılacaktır. Burada max_len parametresi için herhangi bir sayı belirtmemiş olsaydık, tüm diziler en uzun bireysel dizinin uzunluğuna kadar doldurulacaktı. Bizim durumumuzda bu, eğitim süresini önemli ölçüde artıracaktı. Bu arada, daha sonra hiperparametre ayarlama kısmına geldiğinizde, farklı maksimum uzunlukları deneyebilir ve en iyi performansı göstereni bulup bulamayacağınıza bakabilirsiniz. 

### Splitting

Now, split X and y into train, validation and test dataset and assign those to corresponding values.

📌 You can use list slicing methods for this.

📌 For this dataset, a 80-10-10 split corresponds to 40000 - 10000 - 10000 number of samples relatively.


In [11]:
# Create the training datasets
X_train = X[:40000]
y_train = y[:40000]
# Create the validation datasets

X_val = X[40000:45000]
y_val = y[40000:45000]

# Create the test datasets
X_test = X[45000:50000]
y_test = y[45000:50000]

To check if that worked out, print the number of samples in each dataset again.

📌 Use f-strings for this.

In [12]:
# Print the number of samples
print(f"X_train: {len(X_train)}")
print(f"y_train: {len(y_train)}")
print(f"X_val: {len(X_val)}")
print(f"y_val: {len(y_val)}")
print(f"X_test: {len(X_test)}")
print(f"y_test: {len(y_test)}")

X_train: 40000
y_train: 40000
X_val: 5000
y_val: 5000
X_test: 5000
y_test: 5000


## Constructing the neural network

That was it for the preprocessing of the data! 

Now we can create our model. First, we start by creating a model object using the Sequential API of Keras.

📌 Use tf.keras.Sequential() to create a model object

In [13]:
model = tf.keras.Sequential() 

### Embedding Layer

For the first layer, we add an embedding layer.

📌 Use tf.keras.layers.Embedding() for the embedding layer.

📌 Use .add() method of the object to add the layer.

In [14]:
# Add an embedding layer and a dropout
model.add(tf.keras.layers.Embedding(input_dim=10000, output_dim=256))
model.add(tf.keras.layers.Dropout(0.7))

Then, we add a LSTM layer and a dense layer; each with a dropout.

📌 Use tf.keras.layers.LSTM() and tf.keras.layers.Dense() to create the layers.

📌 Use .add() method of the object to add the layer.

In [15]:
# Add a LSTM layer with dropout
model.add(tf.keras.layers.LSTM(256))
model.add(tf.keras.layers.Dropout(0.7))

# Add a Dense layer with dropout
model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dropout(0.7))

### Output layer

As the last part of our neural network, we add the output layer. The number of nodes will be one since we are making binary classification. We'll use the sigmoid activation function in the output layer.

📌 Use tf.keras.layers.Dense() to create the layer.

📌 Use .add() method of the object to add the layer.

In [16]:
# Add the output layer
model.add(tf.keras.layers.Dense(1,activation="sigmoid"))

Modelimizi oluşturmaya başlayabiliriz. İlk olarak bir model nesnesi oluşturuyoruz. Ekleyeceğimiz ilk katman, "gömme katmanı" olarak adlandırılan katmandır. Kelime gömme, kelimeleri veya metni sayısal bir şekilde temsil etmek için bir yöntemdir. Kelimeleri vektör uzayında vektörler biçiminde temsil ederek anlamlarını kodlamamızı sağlar. Burada, benzer bir anlama sahip olması beklenen kelimeler birbirine daha yakındır. Bizim durumumuzda, sahip olduğumuz 10000 kelimeyi 256 boyutlu bir düzleme çevireceğiz. Giriş katmanının sahip olduğumuz veri olduğunu unutmayın. Buraya eklediğimiz katman, giriş katmanı değil, ondan sonraki katmandır. input_dim parametresini kullanarak, giriş katmanının giriş olarak 10000 kelime içerdiğini belirtiyoruz, bu, giriş katmanını ayrı ayrı başka bir katman eklemeden, ancak input_dim parametresini kullanarak eklediğimiz anlamına gelir. Gömmeden sonra, dropout ekliyoruz. Sonra, bir LSTM kapısı, yani bir LSTM katmanı ekleyeceğiz. Düğüm sayısını belirtiyoruz. Şimdi LSTM katmanına dropout ekliyoruz. Ve dropout'lu yoğun bir katman. Ve son olarak bir çıktı katmanı ekliyoruz. Çok sınıflı sınıflandırma problemlerinde çıktı katmanındaki düğüm sayısının sınıf sayısına eşit olacağını ve softmax aktivasyon fonksiyonunu kullanacağımızı unutmayın. Bu projede ikili bir sınıflandırma yapmaya çalışıyoruz. Bu yüzden çıktı katmanında sadece bir düğüm ve bir sigmoid aktivasyon fonksiyonu olacak. Sigmoid aktivasyon fonksiyonu ile ağ 0 ile 1 arasında bir değer üretecek. Yani değer 0,5'in altındaysa sınıf 0 olacak, yani negatif yorumların sınıfı ve üstündeyse sınıf pozitif yorumlar için 1 olacak.

### Optimizer

Now we have the structure of our model. To configure the model for training, we'll use the *.compile()* method. Inside the compile method, we have to define the following:
*   "Adam" for optimizer
*   "Binary Crossentropy" for the loss function


📌 Construct the model with the .compile() method.

In [17]:
model.compile(optimizer="adam", loss= "binary_crossentropy", metrics=["accuracy"])

Modeli derleme yöntemini kullanarak oluşturalım. Her zamanki gibi adam optimizer'ı kullanacağız. Bu sefer "İkili Çapraz Entropi" kayıp fonksiyonunu kullanacağız çünkü bu bir ikili sınıflandırma problemi. İkiden fazla sınıfımız olsaydı "Seyrek Kategorik Çapraz Entropi" kullanmak zorunda kalacağımızı unutmayın. 

## Training the model

It's time to train the model. We'll give the X_train and y_train datasets as the first two arguments. These will be used for training. And with the *validation_data* parameter, we'll give the X_val and y_val as a tuple.

📌 Use .fit() method of the model object for the training.

In [19]:
# Train the model for 5 epochs
results = model.fit(X_train , y_train, epochs=5, validation_data=(X_val , y_val ))

Epoch 1/5
 327/1250 ━━━━━━━━━━━━━━━━━━━━ 1:06:29 4s/step - accuracy: 0.6408 - loss: 0.6434

KeyboardInterrupt: 

 Şimdi onu 5 dönem için eğitiyoruz çünkü bu modeli eğitmek çok fazla zaman alıyor çünkü modelin ayarlaması gereken birçok parametre var. En iyi performansı gösteren modeli bulmak için farklı dönem sayıları deniyorsunuz. 

### Visualize the results

After the model is trained, we can create a graph to visualize the change of loss over time. Results are held in:
* results.history["loss"]
* results.history["val_loss"]

📌 Use plt.show() to display the graph.

In [ ]:
# Plot the the training loss
plt.plot(results_history["loss"], label="Train")

# Plot the the validation loss
plt.plot(results.history["val_loss"], label= "Validation")

# Name the x and y axises
plt.xlabel("Epoch")
plt.ylabel("Loss")

# Put legend table
plt.legend()
# Show the plot
plt.show()

Now, do the same thing for accuracy.

📌 Accuracy scores can be found in:
* results.history["accuracy"]
* results.history["val_accuracy"]



In [ ]:
# Plot the the training accuracy
plt.plot(results.history["accuracy"], label= "Train")

# Plot the the validation accuracy

plt.plot(results.history["val_accuracy"], label= "Validation")
# Name the x and y axises
plt.xlabel("Epoch")
plt.ylabel("Accuracy")

# Put legend table
plt.legend()

# Show the plot
plt.show()

Her iki grafikten de aşırı uyumun olmadığını görebiliyoruz. Her iki grafikteki çizgiler iyi bir şekilde birleşiyor. Ancak bu modeli yalnızca 5 dönem için eğittik. Daha uzun bir süre için eğitim daha iyi bir modele mi yoksa yalnızca aşırı uyuma mı yol açar? Bunu hiperparametre ayarlama kısmında anlayacaksınız. 

## Performance evaluation

Let's use the test dataset that we created to evaluate the performance of the model.

📌 Use test_on_batch() method with test dataset as parameter.

In [ ]:
# Evaluate the performance
model.evaluate(X_test, y_test)

 Modeli X_test ve y_test veri kümelerini kullanarak değerlendirelim. Kayıp değeri 0,24 ve doğruluk 0,90'dır. Doğruluk nispeten yüksek olsa da, belki hiperparametre ayarlamasıyla performansını artırabiliriz.

### Try a prediction

Next, we take a sample and make a prediction on it.

📌 Reshape the review to (1, 1024).

📌 Use the .prediction() method of the model object.

In [ ]:
# Make prediction on the reshaped sample
prediction_result= model.predict(X_test[789], reshape(1,1024))

print(f"Label:{y_test[789]} | prediction: {prediction_result}")

Şimdi rastgele tek bir örnek seçelim ve modelin doğru tahmin edip etmediğine bakalım. Tahmin yöntemi veri grupları üzerinde çalışır ancak yalnızca bir örnek üzerinde tahminde bulunduğumuz için örneği yeniden şekillendirmemiz gerekir. İncelemeyi 1024'e 1 olarak yeniden şekillendireceğiz. Her incelemeyi 1024 uzunluğunda olacak şekilde doldurduğumuzu unutmayın. Daha sonra tahmin sonucunu ve örneğin veri kümesinde sahip olduğu etiketi karşılaştırılmak üzere yazdırırız. Modelimizin çıktısı 0.2'dir. Çıktının 0.5'ten küçük olması durumunda 0'a ait olduğunu söylediğimizi unutmayın. Ve sonuç sıfıra çok yakın olduğundan modelin bu incelemeyi "olumsuz" olarak tahmin ettiğini söyleyebiliriz. Etiket de sıfırdır, bu nedenle modelin doğru bir tahmin yaptığı sonucuna varabiliriz.

Modeli ince ayarlamanın zamanı geldi. Ancak ondan önce, başlangıçta belirlediğimiz parametreyi hatırlıyor musunuz? “load_dataset” fonksiyonunun num_words parametresi ve “pad_sequences” fonksiyonunun maxlen parametresiyle oynayarak bunların model eğitimini ve performansını nasıl etkilediğini görebilirsiniz. Bunu yaptıktan sonra, şu parametreleri değiştirin: LSTM katmanındaki birim sayısı, bırakma oranı, tekrarlayan bırakma oranı, optimize edici ve en iyi performans gösteren modeli bulmaya çalışın.